# Inventory Data Exploration

This notebook explores the inventory monitoring data and provides initial insights.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Data

In [ ]:
# Load inventory data
inventory_df = pd.read_csv('../data/inventory_data.csv')
alerts_df = pd.read_csv('../data/alert_history.csv')

print(f"Inventory records: {len(inventory_df)}")
print(f"Alert records: {len(alerts_df)}")

## 2. Data Overview

In [ ]:
# Display first few rows
inventory_df.head()

In [ ]:
# Data info
inventory_df.info()

In [ ]:
# Summary statistics
inventory_df.describe()

## 3. Stock Status Analysis

In [ ]:
# Stock status distribution
status_counts = inventory_df['stock_status'].value_counts()
print(status_counts)

# Visualize
fig = px.pie(values=status_counts.values, 
             names=status_counts.index,
             title='Stock Status Distribution',
             color_discrete_sequence=['#4caf50', '#ffeb3b', '#ffa500', '#ff4b4b'])
fig.show()

## 4. Location Analysis

In [ ]:
# Items per location
location_summary = inventory_df.groupby('location').agg({
    'item_name': 'nunique',
    'current_stock': 'sum',
    'stock_percentage': 'mean'
}).round(2)

location_summary.columns = ['Unique Items', 'Total Stock', 'Avg Stock %']
location_summary

## 5. Category Analysis

In [ ]:
# Category breakdown
category_summary = inventory_df.groupby('category').agg({
    'item_name': 'nunique',
    'current_stock': 'sum',
    'consumption': 'sum',
    'stock_percentage': 'mean'
}).round(2)

category_summary.columns = ['Items', 'Total Stock', 'Total Consumption', 'Avg Stock %']
category_summary

## 6. Critical Items Analysis

In [ ]:
# Convert date column
inventory_df['date'] = pd.to_datetime(inventory_df['date'])

# Get latest snapshot
latest_date = inventory_df['date'].max()
current_inventory = inventory_df[inventory_df['date'] == latest_date]

# Critical and low stock items
critical_items = current_inventory[current_inventory['stock_status'].isin(['CRITICAL', 'LOW'])]
print(f"\nCritical/Low items: {len(critical_items)}")

# Display top critical items
critical_items.nsmallest(10, 'days_until_stockout')[['location', 'item_name', 'current_stock', 'days_until_stockout', 'stock_status']]

## 7. Time Series Analysis

In [ ]:
# Stock trends over time by category
daily_stock = inventory_df.groupby(['date', 'category'])['current_stock'].sum().reset_index()

fig = px.line(daily_stock, 
              x='date', 
              y='current_stock', 
              color='category',
              title='Stock Levels Over Time by Category',
              labels={'current_stock': 'Total Stock', 'date': 'Date'})
fig.show()

## 8. Consumption Patterns

In [ ]:
# Daily consumption by category
daily_consumption = inventory_df.groupby(['date', 'category'])['consumption'].sum().reset_index()

fig = px.area(daily_consumption,
              x='date',
              y='consumption',
              color='category',
              title='Daily Consumption by Category',
              labels={'consumption': 'Consumption', 'date': 'Date'})
fig.show()

## 9. Alert Analysis

In [ ]:
# Alert distribution
if len(alerts_df) > 0:
    alerts_df['alert_date'] = pd.to_datetime(alerts_df['alert_date'])
    
    # Alerts by priority
    priority_counts = alerts_df['priority'].value_counts()
    print("Alerts by Priority:")
    print(priority_counts)
    
    # Alerts by location
    location_alerts = alerts_df.groupby('location').size().sort_values(ascending=False)
    print("\nTop Locations with Alerts:")
    print(location_alerts.head())
else:
    print("No alerts in the dataset")

## 10. Key Insights

### Findings:
1. **Stock Distribution**: Analyze the distribution of stock levels
2. **Critical Items**: Identify items that frequently run low
3. **Location Performance**: Compare locations for inventory health
4. **Consumption Patterns**: Understand demand patterns by category
5. **Alert Frequency**: Track which items generate most alerts

### Recommendations:
1. Increase reorder points for frequently critical items
2. Balance stock across locations
3. Adjust consumption predictions based on patterns
4. Implement proactive restocking for high-demand items